In [3]:
class test1():
    def __init__(self):
        x = 1
        if x==1:
            print('test1 class not used')
            return
        print('test1')
        self.okk()
    
    def okk(self):
        print('okk')

test1()

test1 class not used


In [1]:
i = 1
print(f"decoder_{i}_inputs")
print('decoder_%s_inputs' % str(i))

decoder_1_inputs
decoder_1_inputs


In [8]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf


class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


class Encoder(layers.Layer):
    """Maps MNIST digits to a triplet (z_mean, z_log_var, z)."""

    def __init__(self, latent_dim=32, intermediate_dim=64, name="encoder", **kwargs):
        super().__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z


class Decoder(layers.Layer):
    """Converts z, the encoded digit vector, back into a readable digit."""

    def __init__(self, original_dim, intermediate_dim=64, name="decoder", **kwargs):
        super().__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation="relu")
        self.dense_output = layers.Dense(original_dim, activation="sigmoid")

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)


class VariationalAutoEncoder(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(
        self,
        original_dim,
        intermediate_dim=64,
        latent_dim=32,
        name="autoencoder",
        **kwargs
    ):
        super().__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.encoder = Encoder(latent_dim=latent_dim, intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # Add KL divergence regularization loss.
        kl_loss = -0.5 * tf.reduce_mean(
            z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1
        )
        self.add_loss(kl_loss)
        return reconstructed

In [11]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype("float32") / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

11490434/11490434 [==============================] - 0s 0us/step


In [13]:
# Iterate over epochs.
for epoch in range(2):
    print("Start of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Compute reconstruction loss
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses)  # Add KLD regularization loss

        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))

        loss_metric(loss)

        if step % 100 == 0:
            print("step %d: mean loss = %.4f" % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = 0.3191
step 100: mean loss = 0.1256
step 200: mean loss = 0.0991
step 300: mean loss = 0.0892
step 400: mean loss = 0.0842
step 500: mean loss = 0.0809
step 600: mean loss = 0.0787
step 700: mean loss = 0.0771
step 800: mean loss = 0.0760
step 900: mean loss = 0.0749
Start of epoch 1
step 0: mean loss = 0.0747
step 100: mean loss = 0.0740
step 200: mean loss = 0.0735
step 300: mean loss = 0.0730
step 400: mean loss = 0.0727
step 500: mean loss = 0.0723
step 600: mean loss = 0.0720
step 700: mean loss = 0.0717
step 800: mean loss = 0.0715
step 900: mean loss = 0.0712


In [14]:
vae.summary()

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  54400     
                                                                 
 decoder (Decoder)           multiple                  53072     
                                                                 
Total params: 107,472
Trainable params: 107,472
Non-trainable params: 0
_________________________________________________________________


In [32]:
class ResnetIdentityBlock(tf.keras.Model):
  def __init__(self, kernel_size):
    #super(ResnetIdentityBlock, self).__init__(name='')
    super().__init__()
    

    self.conv2a = tf.keras.layers.Dense(kernel_size, activation='relu')
    self.bn2a = tf.keras.layers.BatchNormalization()

    self.conv2b = tf.keras.layers.Dense(kernel_size, activation='relu')
    self.bn2b = tf.keras.layers.BatchNormalization()

    self.conv2c = tf.keras.layers.Dense(kernel_size, activation='relu')
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)

    x = self.conv2c(x)
    x = self.bn2c(x, training=training)

    x += input_tensor
    return tf.nn.relu(x)


block = ResnetIdentityBlock(1)

In [33]:
_ = block(tf.zeros([1, 2, 3]))

In [34]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_91 (Dense)            multiple                  4         
                                                                 
 batch_normalization_81 (Bat  multiple                 4         
 chNormalization)                                                
                                                                 
 dense_92 (Dense)            multiple                  2         
                                                                 
 batch_normalization_82 (Bat  multiple                 4         
 chNormalization)                                                
                                                                 
 dense_93 (Dense)            multiple                  2         
                                                                 
 batch_normalization_83 (Bat  multiple       

In [35]:
encoder_inputs = tf.keras.layers.Input(shape=(96,12), name='encoder_past_inputs')

model_output = tf.keras.layers.Dense(12, activation='sigmoid', name='model_output')(encoder_inputs)

for i in range(10):    
    csmodel = ResnetIdentityBlock(12)
    model_output1 = csmodel(model_output)
    model_output = model_output1


model_output3 = tf.keras.layers.Dense(12, activation='sigmoid', name='model_output3')(model_output1)

model = tf.keras.Model(inputs=encoder_inputs, outputs=model_output3)

In [36]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_past_inputs (InputL  [(None, 96, 12)]         0         
 ayer)                                                           
                                                                 
 model_output (Dense)        (None, 96, 12)            156       
                                                                 
 resnet_identity_block_1 (Re  (None, 96, 12)           612       
 snetIdentityBlock)                                              
                                                                 
 resnet_identity_block_2 (Re  (None, 96, 12)           612       
 snetIdentityBlock)                                              
                                                                 
 resnet_identity_block_3 (Re  (None, 96, 12)           612       
 snetIdentityBlock)                                        